# Backtesting assets and a benchmark both b&h and MACD strategy

### In this test we are using vectorbt and pandas_ta (pandas_ta is optional, but is useful

In [ ]:
# Define functions to connect to Google and change directories
def connectDrive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

def changeDirectory(path):
    import os
    original_path = os.getcwd()
    os.chdir(path)
    new_path = os.getcwd()
    print("Original path: ",original_path)
    print("New path: ",new_path)

# Connect to Google Drive
connectDrive()

# Change path
changeDirectory("/content/drive/My Drive/github/find_p/")

In [ ]:
# !pip install vectorbt
# !pip install pandas_ta
# !pip install yfinance
# !pip install plotly
# !pip install numba
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca

In [1]:
import asyncio
import itertools
from datetime import datetime

from IPython import display

import numpy as np
import pandas as pd

import vectorbt as vbt

from pdta_vt_utils import *

import plotly.graph_objects as go
%matplotlib inline

# Configuration of the backtesting platform

### Important step that need check before any experiment

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings


Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Setting of the assets to test

### It is not dynamic and do not offer any advantage but is a way to have the data collected

In [3]:
benchmark_tickers = ["SPY", "QQQ"]
asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]
all_tickers = benchmark_tickers + asset_tickers

print("Tickers by index #")
print("="*100)
print(f"Benchmarks: {', '.join([f'{k}: {v}' for k,v in enumerate(benchmark_tickers)])}")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")
print(f"       All: {', '.join([f'{k}: {v}' for k,v in enumerate(all_tickers)])}")
print("="*100)
benchmarks = dl(benchmark_tickers, lc_cols=True)
assets = dl(asset_tickers, lc_cols=True)

Tickers by index #
Benchmarks: 0: SPY, 1: QQQ
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL, 4: ^GSPC, 5: SPY
       All: 0: SPY, 1: QQQ, 2: AAPL, 3: TSLA, 4: TWTR, 5: SPXL, 6: ^GSPC, 7: SPY


In [5]:
########### You need to set this #############################
# Numero del indice que se quiere ver tanto para exploracion com para benchmark e.j. spy i=5
benchmark_name = benchmark_tickers[0] # Change index for different benchmark
asset_name = asset_tickers[3] # Change index for different symbol
#################################################################

print("="*100)
print(f"Selected Benchmark | Asset: {benchmark_name} | {asset_name}")
start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')
print("Available Data:")
print("="*100)
print(f"Benchmarks: {', '.join(benchmarks.keys())}")
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)

benchmarkdf = benchmarks[benchmark_name]
assetdf     = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    benchmarkdf = dtmask(benchmarkdf, start_date, end_date)
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
benchmarkdf.name = benchmark_name
assetdf.name = asset_name
print(f"Analysis of: {benchmarkdf.name} and {assetdf.name}{crs}")

Available Data:
Benchmarks: SPY, QQQ
Assets: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY


In [7]:
benchmarkdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,87.704898,88.501006,87.033666,88.454178,118944600,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,88.399588,88.727397,88.079580,88.688370,111579900,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,88.602481,88.969318,88.532239,88.750778,116074400,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,88.586849,89.234666,88.337089,89.125397,131091100,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,88.891279,89.461047,88.711767,89.422020,126402800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,179.700982,179.977411,179.441820,179.485016,42963400,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,179.951506,180.418000,179.899676,180.063812,57326700,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,179.873823,180.521722,179.804713,180.305756,79643900,0.0,0.0,0.0


In [8]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,4.100746,4.207280,4.099986,4.200431,28238400,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,4.195866,4.242284,4.147166,4.240001,33206400,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,4.227826,4.278810,4.217934,4.251415,44194800,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,4.229348,4.315334,4.188257,4.301637,43773600,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,4.264352,4.352622,4.240001,4.347295,39685200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372381,21.452076,21.266903,21.290342,1291600,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,21.445038,21.606773,21.421599,21.480198,2223200,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,21.400508,21.634906,21.391132,21.559898,2677600,0.0,0.0,0.0


In [9]:
# Alternative MACD implementation
# def macd_components_gen(dfi):
#     dfi['EMA12'] = dfi.Close.ewm(span=12).mean()
#     dfi['EMA26'] = dfi.Close.ewm(span=26).mean()
#     dfi['MACD'] = dfi.EMA12 - dfi.EMA26
#     dfi['signal'] = dfi.MACD.ewm(span=9).mean()
#     print('Components Generated')

def trends(df: pd.DataFrame, fast: int = 30, slow: int = 15, signal: int = 9):
    macd_ind = vbt.MACD.run(df['Close'],
                            fast_window=fast,
                            slow_window=slow,
                            signal_window=signal)
    entries =  macd_ind.macd_above(0) & macd_ind.macd_above(macd_ind.signal)
    exits = macd_ind.macd_below(0) | macd_ind.macd_below(macd_ind.signal)
    df['TS_Entries'] = entries
    df['TS_Exits'] = exits
    df['TS_Trend'] = entries | exits
    return df, macd_ind

In [10]:
trend_kwargs = {"fast": 39, "slow": 11, "signal": 22}
sl_stop = 0.0688
benchmark_trends, _ = trends(benchmarkdf, **trend_kwargs)
# benchmark_trends.TS_Trend.copy().astype(int).plot(figsize=(16, 1), kind="area", color=["green"], alpha=0.45, title=f"{benchmarkdf.name} Trends", grid=True)

asset_trends, _ = trends(assetdf, **trend_kwargs)
# asset_trends.TS_Trend.copy().astype(int).plot(figsize=(16, 1), kind="area", color=["green"], alpha=0.45, title=f"{assetdf.name} Trends", grid=True)

In [11]:
# # trade_offset = 0 for Live Signals (close is last price)
# # trade_offset = 1 for Backtesting
LIVE = 0

# Backtest the strategy by signals

### Signal Portfolios with their Last 'k' Trades and Performance Statistics

In [12]:
# Benchmark Portfolio from Trade Signals
benchmarkpf_signals = vbt.Portfolio.from_signals(benchmarkdf.Close,
                                                 entries=benchmark_trends.TS_Entries,
                                                 exits=benchmark_trends.TS_Exits,
                                                 sl_stop=sl_stop)
trade_table(benchmarkpf_signals, k=5)
combine_stats(benchmarkpf_signals, benchmarkdf.name, "Long Strategy", LIVE)


Last 5 of 17 Trades
    status  direction      size  entry_price  exit_price    return        pnl  \
12       1          0  7.787917   151.350700  155.599391  0.023002  27.112202   
13       1          0  7.672379   157.154816  159.384582  0.009153  11.036084   
14       1          0  7.389776   164.654506  169.906582  0.026818  32.630831   
15       1          0  7.381146   169.256819  169.775288 -0.001944  -2.429214   
16       1          0  6.971637   178.851271  179.854991  0.000598   0.745646   

    entry_fees  exit_fees  
12    2.946767   3.029488  
13    3.014378   3.057147  
14    3.041900   3.138929  
15    3.123273   3.132841  
16    3.117215   3.134709  



Run Time                      Monday February 13, 2023, NYSE: 14:45:04
Mode                                                              TEST
Strategy                                                 Long Strategy
Direction                                                     longonly
Symbol                                                             SPY
Fees [%]                                                          0.25
Slippage [%]                                                      0.25
Accumulate                                                       False
Start                                        2010-01-04 00:00:00-05:00
End                                          2014-12-31 00:00:00-05:00
Period                                              1258 days 00:00:00
Start Value                                                     1000.0
End Value                                                  1250.749016
Total Return [%]                                             25.074902
Benchm

In [13]:
# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(assetdf.Close,
                                             entries=asset_trends.TS_Entries,
                                             exits=asset_trends.TS_Exits,
                                             sl_stop=sl_stop)
trade_table(assetpf_signals, k=5)
combine_stats(assetpf_signals, assetdf.name, "Long Strategy", LIVE)


Last 5 of 29 Trades
    status  direction       size  entry_price  exit_price    return  \
24       1          0  71.121346    14.782822   15.546136  0.046506   
25       1          0  65.626651    16.763732   18.588027  0.103552   
26       1          0  69.306396    17.513330   15.575724 -0.115360   
27       1          0  68.315834    15.722757   18.281736  0.157349   
28       1          0  59.347612    20.939393   21.159955  0.005507   

           pnl  entry_fees  exit_fees  
24   48.895342    2.628436   2.764155  
25  113.922265    2.750369   3.049675  
26 -140.021729    3.034464   2.698743  
27  169.011169    2.685283   3.122330  
28    6.843641    3.106757   3.139482  



Run Time                      Monday February 13, 2023, NYSE: 14:45:08
Mode                                                              TEST
Strategy                                                 Long Strategy
Direction                                                     longonly
Symbol                                                            SPXL
Fees [%]                                                          0.25
Slippage [%]                                                      0.25
Accumulate                                                       False
Start                                        2010-01-04 00:00:00-05:00
End                                          2014-12-31 00:00:00-05:00
Period                                              1258 days 00:00:00
Start Value                                                     1000.0
End Value                                                  1252.653335
Total Return [%]                                             25.265333
Benchm

# Signal Plots

### Benchmark

In [15]:
assetpf_signals.plot().show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'eb9a4beb-1e05-4791-b4fd-83c84885f126',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 4.20043087,  4.24000072,  4.25141525, ..., 21.55989838, 21.2129879 ,
                          20.56136131]),
              'yaxis': 'y'},
             {'customdata': array([[  0.        , 253.30962678,   2.49376559],
                                   [  2.        , 241.72299417,   2.21501201],
                                   [  4.        , 233.32399187,   1.97225178],
                                   [  6.        , 281.10559042,   2.26513789],
                                   [  8.        , 183.84240633,   2.71907759],
                                   [ 10.        , 186.4492601 ,   3.02598551],
                                   [ 12.        , 185.25043068,   2.77218763],
                                   [ 14.        , 250.4843876 ,   2.96260652],
                                   [ 16.        , 217.46301271,   2.32804191],
                                   [ 18.        , 189.24888863,   2.00042611],
                                   [ 20.        , 197.91301713,   1.82033519],
                                   [ 22.        , 238.26098225,   2.22083497],
                                   [ 24.        , 263.75357417,   2.50545192],
                                   [ 26.        , 179.59599245,   2.88346016],
                                   [ 28.        , 181.53698524,   2.58686635],
                                   [ 30.        , 170.91863928,   2.34385068],
                                   [ 32.        , 170.60365284,   2.12171434],
                                   [ 34.        , 137.00294222,   2.28251576],
                                   [ 36.        , 135.58327192,   2.09478551],
                                   [ 38.        ,  88.08253144,   2.25452466],
                                   [ 40.        ,  86.60719899,   2.06768964],
                                   [ 42.        ,  88.40334443,   2.37906685],
                                   [ 44.        ,  74.32653185,   2.53381039],
                                   [ 46.        ,  72.26112173,   2.30166399],
                                   [ 48.        ,  71.12134562,   2.62843553],
                                   [ 50.        ,  65.62665106,   2.75036905],
                                   [ 52.        ,  69.30639581,   3.03446447],
                                   [ 54.        ,  68.31583371,   2.6852831 ],
                                   [ 56.        ,  59.34761163,   3.10675734]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'legendgroup': '1',
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy',
              'type':

# Just Continue runing if you are satisfied with the backtest, if not run first the optimization and find the good parameters for the indicator.

In [16]:
benchmarkpf_signals.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'cc267396-f165-479b-afad-6b435ee228fa',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([ 88.45417786,  88.68836975,  88.7507782 , ..., 180.30575562,
                          179.33821106, 177.55865479])},
             {'customdata': array([[0, 0, 11.411266366603922, 2.493765586034897, 'Long'],
                                   [1, 1, 11.933603747404584, 2.4853733434297283, 'Long'],
                                   [2, 2, 10.436924019017942, 2.6266603750364084, 'Long'],
                                   [3, 3, 10.392395058005981, 2.7560955301551076, 'Long'],
                                   [4, 4, 11.290851187793846, 2.733500589880805, 'Long'],
                                   [5, 5, 11.972320471122904, 2.721604472506897, 'Long'],
                                   [6, 6, 11.93240462073319, 2.890484975982872, 'Long'],
                                   [7, 7, 10.473748975295251, 2.9692985111878443, 'Long'],
                                   [8, 8, 10.400391182379643, 2.724933836482478, 'Long'],
                                   [9, 9, 9.577006910098628, 2.7993310219976593, 'Long'],
                                   [10, 10, 8.357127350700802, 2.7851956930612687, 'Long'],
                                   [11, 11, 8.314692304010698, 2.911894148134934, 'Long'],
                                   [12, 12, 7.787917084088746, 2.9467667510311912, 'Long'],
                                   [13, 13, 7.672378868632237, 3.014378227442876, 'Long'],
                                   [14, 14, 7.38977560872683, 3.0418996349319514, 'Long'],
                                   [15, 15, 7.381146114426586, 3.1232732776477405, 'Long'],
                                   [16, 16, 6.971637077952794, 3.117215386506132, 'Long']], dtype=object),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'br>Direction: %{customdata[4]}'),
              'marker': {'color': '#4285F4',
                         'line': {'color': 'rgb(11,84,205)', 'width': 1},
                         'size': 7,
                         'symbol': 'square'},
              'mode': 'markers',
              'name': 'Entry',
              'type': 'scatter',
              'uid': '6ff88753-1b91-4b2e-9eeb-1928f81edec6',
              'x': array([datetime.datetime(2010, 5, 7, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2010, 8, 24, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 3, 16, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 5, 25, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 8, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 10, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                      

In [17]:
benchmarkpf_signals.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '¤',
              'showlegend': True,
              'type': 'scatter',
              'uid': '91bc19a2-dc01-4e61-bc0a-652ad2eee08d',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([1000.        , 1000.        , 1000.        , ..., 1250.74901617,
                          1250.74901617, 1250.74901617])}],
    'layout': {'height': 250,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'title': {'text': 'SPY | Equity Curve'},
               'width': 1000}
})

In [18]:
benchmarkpf_signals.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '%',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ec6df5f3-fe9b-4321-9f51-22ae4f2bbe1b',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([ 0.        ,  0.        ,  0.        , ..., -0.00787533, -0.00787533,
                          -0.00787533])}],
    'layout': {'height': 250,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'title': {'text': 'SPY | Drawdown'},
               'width': 1000}
})

In [19]:
benchmarkpf_signals.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'customdata': array([[1.00000000e+00, 1.00000000e+00, 5.66560997e+01, 5.69895262e-02],
                                   [2.00000000e+00, 2.00000000e+00, 5.19034972e+01, 4.94006550e-02],
                                   [5.00000000e+00, 5.00000000e+00, 6.77210819e+01, 6.22069468e-02],
                                   [6.00000000e+00, 6.00000000e+00, 3.16042276e+01, 2.73347102e-02],
                                   [8.00000000e+00, 8.00000000e+00, 2.98332714e+01, 2.73706383e-02],
                                   [1.00000000e+01, 1.00000000e+01, 5.08060805e+01, 4.56036901e-02],
                                   [1.10000000e+01, 1.10000000e+01, 1.39839138e+01, 1.20058569e-02],
                                   [1.20000000e+01, 1.20000000e+01, 2.71122020e+01, 2.30016526e-02],
                                   [1.30000000e+01, 1.30000000e+01, 1.10360844e+01, 9.15286966e-03],
                                   [1.40000000e+01, 1.40000000e+01, 3.26308307e+01, 2.68178068e-02],
                                   [1.60000000e+01, 1.60000000e+01, 7.45646185e-01, 5.98006628e-04]]),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'r>Return: %{customdata[3]:.2%}'),
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'opacity': array([0.85327507, 0.83937686, 0.86283022, 0.79896543, 0.79903123, 0.83242312,
                                           0.77089226, 0.79102989, 0.76566732, 0.79801877, 0.75      ]),
                         'size': array([11.81950305, 11.17091993, 12.26541007,  9.28505323,  9.28812384,
                                        10.8464122 ,  7.97497229,  8.91472833,  7.73114165,  9.24087607,
                                         7.        ]),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Closed - Profit',
              'type': 'scatter',
              'uid': 'fdfdea67-0872-4df4-9f9b-3c46c2d1bf57',
              'x': array([datetime.datetime(2010, 9, 14, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 4, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 10, 13, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2011, 12, 12, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2012, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2013, 7, 11, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2013, 9, 16, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 2, 20, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 4, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 8, 22, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([0.05698953, 0.04940066, 0.06220695, 0.02733471, 0.02737064, 0.04560369,
                          0.01200586, 0.02300165, 0.00915287, 0.02681781, 0.00059801])},
             {'customdata': array([[ 0.00000000e+00,  0.00000000e+00, -3.36528928e+00, -3.37370251e-03],
                                   [ 3.00000000e+00,  3.00000000e+00, -9.06057105e+00, -8.21866564e-03],
                                   [ 4.00000000e+00,  4.00000000e+00, -4.77034307e+00, -4.36285169e-03],
        

In [20]:
benchmarkpf_signals.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '%',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e27fe9cb-6ea4-490a-b89e-2569bca6509f',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([ 0.        ,  0.        ,  0.        , ..., -0.00365681,  0.        ,
                           0.        ])}],
    'layout': {'height': 250,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'title': {'text': 'SPY | Active Returns'},
               'width': 1000}
})

In [21]:
benchmarkpf_signals.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '¤',
              'showlegend': True,
              'type': 'scatter',
              'uid': '67febad5-8ae5-4dd6-906c-d4631291eb1c',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([1000.        , 1000.        , 1000.        , ..., 1250.74901617,
                          1250.74901617, 1250.74901617])}],
    'layout': {'height': 250,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'title': {'text': 'SPY | Cash'},
               'width': 1000}
})

In [22]:
total_assetfees = benchmarkpf_signals.trades.records_readable["Entry Fees"] + benchmarkpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '¤',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9aac3c90-bfae-4aba-b8d8-f11ca856ca89',
              'x': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
              'y': array([4.99159694, 5.12519994, 5.39657092, 5.50329788, 5.46874719, 5.62657809,
                          5.87466719, 5.70789116, 5.53829659, 5.5984876 , 5.7116858 , 5.87343166,
                          5.97625464, 6.07152548, 6.18082842, 6.2561138 , 6.2519247 ])}],
    'layout': {'height': 250,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'title': {'text': 'SPY | Total Fees'},
               'width': 1000}
})

### Asset

In [ ]:
assetpf_signals.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()

In [ ]:
assetpf_signals.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
total_assetfees = assetpf_signals.trades.records_readable["Entry Fees"] + assetpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

# Uncomment and run the follwing just if you want a benchmark with buy and hold in details, remember the first plot containg the buy and hold of the Asset

In [ ]:
# Benchmark Buy and Hold (BnH) Strategy
# benchmarkpf_bnh = vbt.Portfolio.from_holding(benchmarkdf.Close)
# print(trade_table(benchmarkpf_bnh))
# combine_stats(benchmarkpf_bnh, benchmarkdf.name, "Buy and Hold", LIVE)
#### Benchmark Buy and Hold Plots #######################
# vbt.settings.set_theme("seaborn")
# benchmarkpf_bnh.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()
# benchmarkpf_bnh.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = benchmarkpf_bnh.trades.records_readable["Entry Fees"] + benchmarkpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
# Asset Buy and Hold (BnH) Strategy
# assetpf_bnh = vbt.Portfolio.from_holding(assetdf.Close)
# print(trade_table(assetpf_bnh))
# combine_stats(assetpf_bnh, assetdf.name, "Buy and Hold", LIVE)
### Asset Buy and Hold ############################
# vbt.settings.set_theme("seaborn")
# assetpf_bnh.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()
# assetpf_bnh.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = assetpf_bnh.trades.records_readable["Entry Fees"] + assetpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()